# Single Agent MCP Example

To start, login to azure via the CLI:

`az login --tenant <YOUR_TENANT_ID>`

`az account set -s <YOUR_SUBSCRIPTION_ID>`

In [ ]:
import os, time
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval

In [ ]:
load_dotenv()

agent_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential()
)

In [ ]:
# Get the MCP server configuration from environment variables
mcp_server_url = os.environ.get("MCP_MSFT_DOCS_SERVER")
mcp_server_label = "msdocs"

print(f"Using MCP server: {mcp_server_label} at {mcp_server_url}")

mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
)

In [ ]:
agent_id = ""
# Create agent with MCP tool and process agent run

    # Create a new agent.
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
agent = agent_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="azure-expert-agent",
        instructions=(
            """
            You are an expert in Azure services and tools. Your task is to assist users with their Azure-related questions and tasks.
            
            Use the MCP tool to: 
            - Access the latest Azure documentation and resources to provide accurate and up-to-date information.
            - Share useful links, code snippets, and explanations based on the user's request.
            - Perform tasks such as retrieving documentation, checking service status, and providing guidance on Azure best practices.

            When a user asks a question or requests assistance, you will use the MCP tool to find relevant information and provide a response. 
            If the MCP tool is not available, you will inform the user that the tool is not available and cannot assist with their request.
            Ensure that your responses are clear, concise, and relevant to the user's request. Do not provide any unverified information.
            If you are unable to assist with a request, inform the user that you cannot help with that request at this time.

            """
        ),
        tools=mcp_tool.definitions,
    )
agent_id = agent.id
print(f"Created agent: {agent.name} with ID: {agent.id}")

In [ ]:
# create agent thread
thread = agent_client.threads.create()
print(f"Created thread, ID: {thread.id}")

In [ ]:
question = """
I have a customer that is interested in building AI solutions on Azure, but they do not have a strong developer background. 
What are the best low-code solutions available in Azure and M365 to help them get started with AI?
"""

#question = """
#I have a customer that runs Ansys on-premises and wants to migrate to Azure. They run the workload using a Slurm cluster.
#What is the best platform and architecture I can recommend for this migration?
#"""

In [ ]:
# Create a message for the thread
message = agent_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=question,
)
print(f"Created message, ID: {message.id}")


In [ ]:
mcp_tool.set_approval_mode("never")  
run = agent_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

In [ ]:
while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agent_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agent_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agent_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")
        # [END handle_tool_approvals]

print(f"Run completed with status: {run.status}")
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

In [ ]:
run_steps = agent_client.run_steps.list(thread_id=thread.id, run_id=run.id)

for step in run_steps:
    print(f"Step ID: {step.id}, Status: {step.status}")

    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])
    
    if tool_calls:
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")
    print()

In [ ]:
messages = agent_client.messages.list(thread_id=thread.id)

print("\nConversation:")
print("-" * 50)

for msg in messages:
    if msg.text_messages: 
        last_text = msg.text_messages[-1]
        print(f"{msg.role.upper()}: {last_text.text.value}")
        print("-" * 50)

In [ ]:
agent_client.delete_agent(agent_id)
print("Deleted agent")